In [ ]:
%pip install sentence-transformers

In [ ]:
%pip install replicate
%pip install pypdf
%pip install langchain==0.2.0
%pip install langchain-pinecone==0.0.7
%pip install -U langchain-community

In [16]:
import pinecone
import replicate
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community .llms import CTransformers
from langchain_pinecone import PineconeVectorStore

c:\Users\Admin\Desktop\Trial file\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [17]:
PINECONE_API_KEY = "c85a2244-4db3-464b-a9e2-f366e296718d"
PINECONE_API_ENV = "us-east-1"

In [76]:
def load_pdf(data):    
        loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader)
        
        documents = loader.load()
        return documents
        print(documents)
        

In [77]:
extracted_data = load_pdf("./data/")

In [ ]:
extracted_data

In [78]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [79]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 1911


In [ ]:
text_chunks

In [82]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [83]:
embeddings = download_hugging_face_embeddings()

In [84]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [85]:
#Initializing the Pinecone
import os
from pinecone import Pinecone

os.environ['PINECONE_API_KEY']= 'c85a2244-4db3-464b-a9e2-f366e296718d'

pinecone.instance = Pinecone(api_key='c85a2244-4db3-464b-a9e2-f366e296718d',environment='us-east-1')

index_name="graduating-project"

from langchain_pinecone import PineconeVectorStore

#Creating Embeddings for Each of The Text Chunks & storing
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [38]:
#If we already have an index we can load it like this
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "How to manage a farm"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='One fina l important bit of advice : Remember to pass on what you know to a\nfuture sm all farm er . T eaching others is also a great way of reinf orcing what\nyou kno w . By being honest about the joys and pitfalls of farmi ng, you can\nmentor others about the wonderful world of small-scale farming.\nT able of Contents'), Document(page_content='clearly\tmakes\tsense\tand\tis\tworth\trepeating—go\tand\twork\ton\ta\tfarm.\tTry\tout\nthe\tidea\tby\tlaboring\t(and\tlearning)\twith\tsomeone\telse.\tExperience\tthe\tgood\ntimes\tand\tthe\tbad,\tthe\trealities\tand\tthe\trewards.\tIf\tpossible,\twork\ton\tmore\nthan\tone\tfarm.\tThe\tmore\tbackground\tand\texperience\tyou\tcan\tget,\tthe\tbetter\noff\tyou’ll\tbe.\nThe\trequirements\tfor\tsuccess\tin\tfarming\tare\tlike\tthose\tfor\tany\tsmall\nbusiness:\torganizational\taptitude,\tdiligence,\tfinancial\tplanning,\tthe\tability\tto'), Document(page_content='the\toperation\tshould\tspend\this\tor\ther\ttime\tdoin

In [ ]:
def load_pinecone_existing_index():
    docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)
    return docsearch
docsearch=load_pinecone_existing_index()

query = "How to manage a farm"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='A few of the recipes in this book call for specialty flours like almond,\nwhole wheat, and spelt. Luckily , these can now be found fairly easily at\nregular grocery stores. They add an extra element of flavor and/or texture,\nbut can usually be substituted.\nTo meas ure flour, use the scoop-and-sweep method. First, aerate the\nflour in its container by briefly mixing the contents with a whisk. Then\nscoop flour into the measuring cup until it is slightly overflowing. Level off'), Document(page_content='westelm.com\nBaking tools, dinnerware, flatware, linens, and more\nWhole Foods Market:\nwholefoodsmarket.com\nOrganic and specialty foods\nWilliams-Sonoma:\nwilliams-sonoma.com\nEquipment, baking tools, cake pans, and some specialty foods'), Document(page_content='British India, described it as “the embodiment of all things pure.”')]


In [39]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [40]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [41]:
from langchain.llms import Replicate
os.environ['REPLICATE_API_TOKEN']="r8_YgNQpNUeSV2XL230IieDlrWC1YU0KFR3eQjJz"
llm = Replicate(model)
# llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
#                   model_type="llama",
#                   config={'max_new_tokens':512,
#                           'temperature':0.8})

SyntaxError: invalid syntax (4002402046.py, line 3)

In [ ]:
retriever=docsearch.as_retriever(search_kwargs={'k': 2})
print(retriever)

tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001C6918C60F0> search_kwargs={'k': 2}


In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

: 